# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### --PENDING-- US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'08-03-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'08-03-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-08-04 04:41:59,34.223334,-82.461707,298,7,0,291.0,"Abbeville, South Carolina, US",1214.987565,2.348993
1,22001.0,Acadia,Louisiana,US,2020-08-04 04:41:59,30.295065,-92.414197,2403,77,0,2326.0,"Acadia, Louisiana, US",3872.995407,3.204328
2,51001.0,Accomack,Virginia,US,2020-08-04 04:41:59,37.767072,-75.632346,1080,15,0,1065.0,"Accomack, Virginia, US",3341.997772,1.388889
3,16001.0,Ada,Idaho,US,2020-08-04 04:41:59,43.452658,-116.241552,8069,64,0,8005.0,"Ada, Idaho, US",1675.502038,0.793159
4,19001.0,Adair,Iowa,US,2020-08-04 04:41:59,41.330756,-94.471059,21,0,0,21.0,"Adair, Iowa, US",293.624161,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,36036,36157,36263,36368,36471,36542,36675,36710,36710,36747
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,4637,4763,4880,4997,5105,5197,5276,5396,5519,5620
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,26764,27357,27973,28615,29229,29831,30394,30950,31465,31972


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1248,1259,1269,1270,1271,1271,1272,1283,1284,1288
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,134,138,144,148,150,154,157,161,166,172
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1146,1155,1163,1174,1186,1200,1210,1223,1231,1239


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,24793,25180,25198,25358,25389,25471,25509,25509,25510,25669
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2637,2682,2745,2789,2830,2883,2952,2961,3018,3031
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,18076,18088,18837,19233,19592,20082,20537,20988,21419,21901


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,337,337,349,351,354,356,356,356,368,375
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,40,40,40,40,42,42,42,44,45,46


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
1172,1065.0,Hale,Alabama,US,2020-08-04 04:41:59,32.760393,-87.632850,459,26,0,433.0,"Hale, Alabama, US",3132.891953,5.664488
2291,1107.0,Pickens,Alabama,US,2020-08-04 04:41:59,33.280949,-88.088181,367,9,0,358.0,"Pickens, Alabama, US",1841.445058,2.452316
651,1037.0,Coosa,Alabama,US,2020-08-04 04:41:59,32.936901,-86.248477,92,2,0,90.0,"Coosa, Alabama, US",862.796586,2.173913


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,4713540,155402,1513446
Brazil,2750318,94665,2098976
India,1855745,38938,1230509


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,4713540,155402,1513446,3044692,2020-08-04 04:41:59,37.904997,-91.310033
Brazil,2750318,94665,2098976,556677,2020-08-04 04:41:59,-12.669522,-48.480493
India,1855745,38938,1230509,586298,2020-08-04 04:41:59,23.088275,81.806127
Russia,854641,14183,652372,188086,2020-08-04 04:41:59,54.546312,62.120860
South Africa,516862,8539,358037,150286,2020-08-04 04:41:59,-30.559500,22.937500


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,516851,9441,0
Florida,491884,7157,0
Texas,456624,7016,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,516851,9441,0,507410,2020-08-04 04:41:59,37.843962,-120.728594
Florida,491884,7157,0,484727,2020-08-04 04:41:59,28.940755,-82.700744
Texas,456624,7016,0,449608,2020-08-04 04:41:59,31.647790,-98.615963
New York,416843,32719,0,384124,2020-08-04 04:41:59,42.671593,-75.579694
Georgia,195435,3842,0,191593,2020-08-04 04:41:59,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,225964,23550,0
California,Los Angeles,193877,4702,0
Florida,Miami-Dade,123644,1694,0
Arizona,Maricopa,120960,2108,0
Illinois,Cook,107247,4897,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,225964,23550,0,202414,2020-08-04 04:41:59,40.767273,-73.971526,36061.0
California,Los Angeles,193877,4702,0,189175,2020-08-04 04:41:59,34.308284,-118.228241,6037.0
Florida,Miami-Dade,123644,1694,0,121950,2020-08-04 04:41:59,25.611236,-80.551706,12086.0
Arizona,Maricopa,120960,2108,0,118852,2020-08-04 04:41:59,33.348359,-112.491815,4013.0
Illinois,Cook,107247,4897,0,102350,2020-08-04 04:41:59,41.841448,-87.816588,17031.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-01,36710,5396,30950,925,1164,91,196543,38841,17895,21212,...,305562,1278,24783,19443,590,12160,10,1730,6228,3659
2020-08-02,36710,5519,31465,925,1199,91,201919,39050,18318,21304,...,306309,1286,25336,20206,621,12297,10,1734,6347,3921
2020-08-03,36747,5620,31972,937,1280,92,206743,39102,18730,21385,...,307251,1291,26066,20754,652,12541,10,1734,6580,4075


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-01,1283,161,1223,52,54,3,3596,749,208,718,...,46278,35,147,169,3,83,1,494,165,69
2020-08-02,1284,166,1231,52,55,3,3648,754,221,718,...,46286,36,151,174,6,84,1,497,170,70
2020-08-03,1288,172,1239,52,58,3,3813,762,232,718,...,46295,36,157,180,7,84,1,499,171,80


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-01,25509,2961,20988,807,460,67,86499,29557,10204,18911,...,1441,1004,15299,11129,373,5324,8,862,4130,1011
2020-08-02,25510,3018,21419,807,461,67,89026,29750,10622,18984,...,1444,1011,15833,11404,373,5390,8,862,4493,1016
2020-08-03,25669,3031,21901,821,476,75,91302,29861,10623,19063,...,1445,1023,16838,11622,374,5390,8,863,4701,1057


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20
130,84001097,US,USA,840,1097.0,Mobile,Alabama,US,30.784723,-88.208424,...,7096,7193,7361,7495,7683,7879,8084,8466,8998,9086
109,84001055,US,USA,840,1055.0,Etowah,Alabama,US,34.045673,-86.040519,...,1615,1643,1690,1723,1743,1797,1857,1884,1914,1930
110,84001057,US,USA,840,1057.0,Fayette,Alabama,US,33.720769,-87.738866,...,134,138,144,151,156,162,163,167,169,174
111,84001059,US,USA,840,1059.0,Franklin,Alabama,US,34.442353,-87.842895,...,1075,1078,1098,1105,1117,1144,1162,1165,1184,1196
112,84001061,US,USA,840,1061.0,Geneva,Alabama,US,31.093890,-85.835728,...,183,186,192,194,198,204,210,216,226,232


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-01,89349,3136,0,177002,43173,506613,47249,49810,14877,49,...,108184,444738,40702,1421,421,90801,57541,6738,54002,2769
2020-08-02,91444,3280,0,178467,43810,512175,47709,49810,14949,49,...,109627,448145,41175,1426,421,91782,58173,6858,54924,2808
2020-08-03,92661,3341,0,179497,44597,516851,47950,50062,15055,49,...,110636,456624,41529,1427,439,93106,58715,6978,55328,2848


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-01         988    3095     554  360    704     436    743    1438   
2020-08-02        1010    3175     562  364    722     437    751    1535   
2020-08-03        1024    3210     563  374    736     438    752    1560   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-01          807      222  ...     117      5       60       36   
2020-08-02          814      233  ...     117      5       60       37   
2020-08-03          821      242  ...     117      5       61       37   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-01            242   348   259          0       49      5  
2020-08-02            247   354   262          0       49      5  
2020-08-03            250   358   265          0       49      5  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-01,1603,24,0,3747,458,9345,1844,4432,585,0,...,1067,6865,310,57,8,2215,1592,116,947,26
2020-08-02,1627,24,0,3765,464,9396,1844,4432,585,0,...,1073,6878,311,57,8,2218,1596,117,948,26
2020-08-03,1633,25,0,3779,475,9441,1844,4437,585,0,...,1092,7016,314,57,8,2218,1600,117,949,27


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-01          20      21       5    2      3      11     35       9   
2020-08-02          20      22       5    3      3      11     35       9   
2020-08-03          20      23       5    3      3      11     35       9   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-01           38        7  ...       0      0        0        0   
2020-08-02           38        7  ...       0      0        0        0   
2020-08-03           38        7  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-01              0     0     0         25        0      0  
2020-08-02              0     0     0         25        0      0  
2020-08-03              0     0     0         26        0      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-01,0.000954,0.022745,0.018293,0.000000,0.013937,0.000000,0.027396,0.007549,0.035590,0.003881,...,0.002523,0.011076,0.032238,0.046786,0.057348,0.027287,0.0,0.001157,0.044441,0.154623
2020-08-02,0.000000,0.022795,0.016640,0.000000,0.030069,0.000000,0.027353,0.005381,0.023638,0.004337,...,0.002445,0.006260,0.022314,0.039243,0.052542,0.011266,0.0,0.002312,0.019107,0.071604
2020-08-03,0.001008,0.018300,0.016113,0.012973,0.067556,0.010989,0.023891,0.001332,0.022492,0.003802,...,0.003075,0.003888,0.028813,0.027121,0.049919,0.019842,0.0,0.000000,0.036710,0.039276


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-01,0.008648,0.025478,0.010744,0.0,0.038462,0.0,0.014959,0.014905,0.034826,0.0,...,0.001602,0.000000,0.042553,0.030488,0.000000,0.012195,0.0,0.002028,0.092715,0.029851
2020-08-02,0.000779,0.031056,0.006541,0.0,0.018519,0.0,0.014461,0.006676,0.062500,0.0,...,0.000173,0.028571,0.027211,0.029586,1.000000,0.012048,0.0,0.006073,0.030303,0.014493
2020-08-03,0.003115,0.036145,0.006499,0.0,0.054545,0.0,0.045230,0.010610,0.049774,0.0,...,0.000194,0.000000,0.039735,0.034483,0.166667,0.000000,0.0,0.004024,0.005882,0.142857


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-01,0.000000,0.003049,0.021960,0.000000,0.052632,0.000000,0.032454,0.019312,0.022138,0.008157,...,0.001390,0.010060,0.057730,0.067940,0.000000,0.048651,0.0,0.00000,0.085985,0.006972
2020-08-02,0.000039,0.019250,0.020536,0.000000,0.002174,0.000000,0.029214,0.006530,0.040964,0.003860,...,0.002082,0.006972,0.034904,0.024710,0.000000,0.012397,0.0,0.00000,0.087893,0.004946
2020-08-03,0.006233,0.004307,0.022503,0.017348,0.032538,0.119403,0.025566,0.003731,0.000094,0.004161,...,0.000693,0.011869,0.063475,0.019116,0.002681,0.000000,0.0,0.00116,0.046294,0.040354


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-01,0.018536,0.048829,NaN,0.017194,0.015572,0.012101,0.009788,0.000000,0.006018,0.0,...,0.020999,0.014705,0.012588,0.004950,0.036946,0.010157,0.031145,0.024168,0.020060,0.015774
2020-08-02,0.023447,0.045918,NaN,0.008277,0.014755,0.010979,0.009736,0.000000,0.004840,0.0,...,0.013338,0.007661,0.011621,0.003519,0.000000,0.010804,0.010983,0.017809,0.017073,0.014085
2020-08-03,0.013309,0.018598,NaN,0.005771,0.017964,0.009130,0.005051,0.005059,0.007091,0.0,...,0.009204,0.018920,0.008597,0.000701,0.042755,0.014425,0.009317,0.017498,0.007356,0.014245


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-01      0.016461  0.012762  0.007273  0.014085  0.027737  0.013953   
2020-08-02      0.022267  0.025848  0.014440  0.011111  0.025568  0.002294   
2020-08-03      0.013861  0.011024  0.001779  0.027473  0.019391  0.002288   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-08-01      0.005413  0.020582  0.001241  0.013699  ...  0.017391    0.0   
2020-08-02      0.010767  0.067455  0.008674  0.049550  ...  0.000000    0.0   
2020-08-03      0.001332  0.016287  0.008600  0.038627  ...  0.000000    0.0   

Province_State                                                                \
Admin2          Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2020-08-01      0.034483  0.058824   0.008333  0.038806  0.019685        NaN   
2020-08-02      0.000000  0.027778   0.020661  0.017241  0.011583        NaN   
2020-08-03      0.016667  0.000000   0.012146  0.011299  0.011450        NaN   

Province_State                   
Admin2          Washakie Weston  
2020-08-01      0.020833    0.0  
2020-08-02      0.000000    0.0  
2020-08-03      0.000000    0.0  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-01,0.014557,0.043478,NaN,0.014348,0.011038,0.014217,0.003264,0.000000,0.0,NaN,...,0.006604,0.043948,0.019737,0.0,0.0,0.018859,0.017903,0.000000,0.013919,0.000000
2020-08-02,0.014972,0.000000,NaN,0.004804,0.013100,0.005457,0.000000,0.000000,0.0,NaN,...,0.005623,0.001894,0.003226,0.0,0.0,0.001354,0.002513,0.008621,0.001056,0.000000
2020-08-03,0.003688,0.041667,NaN,0.003718,0.023707,0.004789,0.000000,0.001128,0.0,NaN,...,0.017707,0.020064,0.009646,0.0,0.0,0.000000,0.002506,0.000000,0.001055,0.038462


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                       \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-01         0.0  0.000000     0.0  0.0    0.0     0.0    0.0     0.0   
2020-08-02         0.0  0.047619     0.0  0.5    0.0     0.0    0.0     0.0   
2020-08-03         0.0  0.045455     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-01          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-08-02          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-08-03          0.0      0.0  ...     NaN    NaN      NaN      NaN   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-01            NaN   NaN   NaN       0.00      NaN    NaN  
2020-08-02            NaN   NaN   NaN       0.00      NaN    NaN  
2020-08-03            NaN   NaN   NaN       0.04      NaN    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-01,0.001987,0.020268,0.019213,0.002311,0.026630,0.004551,0.030243,0.007842,0.032357,0.005527,...,0.002635,0.012119,0.032082,0.043013,0.069795,0.026302,5.052748e-14,0.002644,0.055194,0.097437
2020-08-02,0.000994,0.021531,0.017926,0.001156,0.028349,0.002275,0.028798,0.006612,0.027998,0.004932,...,0.002540,0.009190,0.027198,0.041128,0.061168,0.018784,2.526374e-14,0.002478,0.037150,0.084521
2020-08-03,0.001001,0.019916,0.017020,0.007064,0.047953,0.006632,0.026344,0.003972,0.025245,0.004367,...,0.002808,0.006539,0.028005,0.034124,0.055544,0.019313,1.263187e-14,0.001239,0.036930,0.061898


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-01,0.004905,0.023879,0.010111,1.393221e-16,0.039062,1.552781e-33,0.025116,0.012730,0.036031,0.00064,...,0.001767,0.000476,0.040898,0.030410,0.000000,0.012865,0.0,0.005107,0.054798,0.124109
2020-08-02,0.002842,0.027468,0.008326,6.966105e-17,0.028790,7.763905e-34,0.019788,0.009703,0.049266,0.00032,...,0.000970,0.014524,0.034054,0.029998,0.666667,0.012456,0.0,0.005590,0.042550,0.069301
2020-08-03,0.002979,0.031806,0.007413,3.483053e-17,0.041668,3.881953e-34,0.032509,0.010157,0.049520,0.00016,...,0.000582,0.007262,0.036895,0.032240,0.380952,0.006228,0.0,0.004807,0.024216,0.106079


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-01,0.001251,0.011925,0.022446,0.000699,0.103396,0.003301,0.034943,0.019866,0.020125,0.007340,...,0.000974,0.011444,0.057295,0.037867,0.001702,0.052442,2.960595e-16,0.004672,0.093547,0.064010
2020-08-02,0.000645,0.015588,0.021491,0.000349,0.052785,0.001651,0.032078,0.013198,0.030545,0.005600,...,0.001528,0.009208,0.046100,0.031288,0.000851,0.032419,1.480297e-16,0.002336,0.090720,0.034478
2020-08-03,0.003439,0.009948,0.021997,0.008849,0.042661,0.060527,0.028822,0.008465,0.015319,0.004881,...,0.001110,0.010539,0.054787,0.025202,0.001766,0.016210,7.401487e-17,0.001748,0.068507,0.037416


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-01,0.020181,0.042008,NaN,0.016896,0.017150,0.015176,0.010784,0.001773,0.006372,9.769734e-43,...,0.023368,0.018745,0.012351,0.004017,0.035434,0.010669,0.019313,0.024929,0.018690,0.016705
2020-08-02,0.021814,0.043963,NaN,0.012586,0.015952,0.013077,0.010260,0.000886,0.005606,4.884867e-43,...,0.018353,0.013203,0.011986,0.003768,0.017717,0.010736,0.015148,0.021369,0.017882,0.015395
2020-08-03,0.017562,0.031280,NaN,0.009179,0.016958,0.011103,0.007656,0.002973,0.006348,2.442433e-43,...,0.013779,0.016062,0.010292,0.002235,0.030236,0.012581,0.012233,0.019434,0.012619,0.014820


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-01      0.013822  0.020904  0.010092  0.020365  0.034161  0.010549   
2020-08-02      0.018044  0.023376  0.012266  0.015738  0.029865  0.006421   
2020-08-03      0.015953  0.017200  0.007023  0.021605  0.024628  0.004355   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-08-01      0.006506  0.042011  0.004258  0.015528  ...  0.024132   
2020-08-02      0.008637  0.054733  0.006466  0.032539  ...  0.012066   
2020-08-03      0.004984  0.035510  0.007533  0.035583  ...  0.006033   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-08-01      0.000122  0.059835  0.052252   0.012221  0.032175  0.019755   
2020-08-02      0.000061  0.029917  0.040015   0.016441  0.024708  0.015669   
2020-08-03      0.000031  0.023292  0.020008   0.014294  0.018004  0.013560   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-08-01           -1.0  0.024077  0.003918  
2020-08-02           -1.0  0.012038  0.001959  
2020-08-03           -1.0  0.006019  0.000980  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-01,0.014278,0.029925,NaN,0.020238,0.016888,0.016523,0.004554,0.000310,0.002077,NaN,...,0.013812,0.037192,0.019002,0.002232,0.009010,0.015388,0.010816,0.013176,0.013044,0.001259
2020-08-02,0.014625,0.014962,NaN,0.012521,0.014994,0.010990,0.002277,0.000155,0.001039,NaN,...,0.009717,0.019543,0.011114,0.001116,0.004505,0.008371,0.006665,0.010898,0.007050,0.000629
2020-08-03,0.009156,0.028315,NaN,0.008120,0.019351,0.007890,0.001138,0.000642,0.000519,NaN,...,0.013712,0.019803,0.010380,0.000558,0.002252,0.004186,0.004585,0.005449,0.004052,0.019545


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                                   \
Admin2               Autauga   Baldwin   Barbour      Bibb   Blount   Bullock   
2020-08-01      1.753898e-06  0.023737  0.031292  0.000002  0.12500  0.000049   
2020-08-02      8.769488e-07  0.035678  0.015646  0.250001  0.06250  0.000024   
2020-08-03      4.384744e-07  0.040566  0.007823  0.125000  0.03125  0.000012   

Province_State                                              ... Wyoming  \
Admin2            Butler   Calhoun  Chambers      Cherokee  ...    Park   
2020-08-01      0.000497  0.072917  0.000611  1.630645e-13  ...     NaN   
2020-08-02      0.000249  0.036459  0.000305  8.153224e-14  ...     NaN   
2020-08-03      0.000124  0.018229  0.000153  4.076612e-14  ...     NaN   

Province_State                                                             \
Admin2         Platte Sheridan Sublette Sweetwater Teton Uinta Unassigned   
2020-08-01        NaN      NaN      NaN        NaN   NaN   NaN   0.001311   
2020-08-02        NaN      NaN      NaN        NaN   NaN   NaN   0.000656   
2020-08-03        NaN      NaN      NaN        NaN   NaN   NaN   0.020328   

Province_State                  
Admin2         Washakie Weston  
2020-08-01          NaN    NaN  
2020-08-02          NaN    NaN  
2020-08-03          NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200803,AK,4064,241743,NaN,38.0,NaN,NaN,NaN,4.0,...,245807,1,0,890afec5cf195d314c578f8709ac457553a5acdb,0,0,0,0,0,NaN
1,20200803,AL,92661,621390,NaN,1517.0,10865.0,NaN,1153.0,NaN,...,714051,6,344,e2c9f30805eee72254984ba76c01fd4d5d4baca4,0,0,0,0,0,NaN
2,20200803,AR,44597,482803,NaN,513.0,2966.0,NaN,NaN,108.0,...,527400,17,72,a5dde28205d14b46b4ca50a97b96cef8c6a3770e,0,0,0,0,0,NaN
3,20200803,AS,0,1267,NaN,NaN,NaN,NaN,NaN,NaN,...,1267,0,0,6ac3b3dfcfdb6433bf96d9b1b7dfae9d28d21891,0,0,0,0,0,NaN
4,20200803,AZ,179497,784921,NaN,2017.0,11400.0,628.0,NaN,461.0,...,964418,14,29,9591393f2723f5245eb0a913e9ba03e05f7ce0bd,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200803,AK,4064.0,241743.0,NaN,38.0,NaN,NaN,NaN,4.0,...,245807,1,0,890afec5cf195d314c578f8709ac457553a5acdb,0,0,0,0,0,NaN
1,20200803,AL,92661.0,621390.0,NaN,1517.0,10865.0,NaN,1153.0,NaN,...,714051,6,344,e2c9f30805eee72254984ba76c01fd4d5d4baca4,0,0,0,0,0,NaN
2,20200803,AR,44597.0,482803.0,NaN,513.0,2966.0,NaN,NaN,108.0,...,527400,17,72,a5dde28205d14b46b4ca50a97b96cef8c6a3770e,0,0,0,0,0,NaN
3,20200803,AS,0.0,1267.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1267,0,0,6ac3b3dfcfdb6433bf96d9b1b7dfae9d28d21891,0,0,0,0,0,NaN
4,20200803,AZ,179497.0,784921.0,NaN,2017.0,11400.0,628.0,NaN,461.0,...,964418,14,29,9591393f2723f5245eb0a913e9ba03e05f7ce0bd,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-03,AK,4064,241743,NaN,38.0,NaN,NaN,NaN,4.0,NaN,...,245807,1,0,890afec5cf195d314c578f8709ac457553a5acdb,0,0,0,0,0,NaN
2020-08-03,AL,92661,621390,NaN,1517.0,10865.0,NaN,1153.0,NaN,619.0,...,714051,6,344,e2c9f30805eee72254984ba76c01fd4d5d4baca4,0,0,0,0,0,NaN
2020-08-03,AR,44597,482803,NaN,513.0,2966.0,NaN,NaN,108.0,403.0,...,527400,17,72,a5dde28205d14b46b4ca50a97b96cef8c6a3770e,0,0,0,0,0,NaN
2020-08-03,AS,0,1267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1267,0,0,6ac3b3dfcfdb6433bf96d9b1b7dfae9d28d21891,0,0,0,0,0,NaN
2020-08-03,AZ,179497,784921,NaN,2017.0,11400.0,628.0,NaN,461.0,NaN,...,964418,14,29,9591393f2723f5245eb0a913e9ba03e05f7ce0bd,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-03,AK,4064.0,241743.0,NaN,38.0,NaN,NaN,NaN,4.0,NaN,...,245807,1,0,890afec5cf195d314c578f8709ac457553a5acdb,0,0,0,0,0,NaN
2020-08-03,AL,92661.0,621390.0,NaN,1517.0,10865.0,NaN,1153.0,NaN,619.0,...,714051,6,344,e2c9f30805eee72254984ba76c01fd4d5d4baca4,0,0,0,0,0,NaN
2020-08-03,AR,44597.0,482803.0,NaN,513.0,2966.0,NaN,NaN,108.0,403.0,...,527400,17,72,a5dde28205d14b46b4ca50a97b96cef8c6a3770e,0,0,0,0,0,NaN
2020-08-03,AS,0.0,1267.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1267,0,0,6ac3b3dfcfdb6433bf96d9b1b7dfae9d28d21891,0,0,0,0,0,NaN
2020-08-03,AZ,179497.0,784921.0,NaN,2017.0,11400.0,628.0,NaN,461.0,NaN,...,964418,14,29,9591393f2723f5245eb0a913e9ba03e05f7ce0bd,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE